In [21]:
import pandas as pd
import numpy as np
import os
import glob
import plotly.express as px
from datasets import load_from_disk

print(os.listdir("Experiments"))
experiment = "Experiments/3_retrieval_fetch_k"
metrics_files = glob.glob(os.path.join(experiment, "metrics_*.csv"))
metrics_files.sort()
metrics_files

['1_anonymize_parties', '.DS_Store', '2_llm', 'testing.ipynb', '3_retrieval_fetch_k']


['Experiments/3_retrieval_fetch_k/metrics_3a_fetch_k_20.csv',
 'Experiments/3_retrieval_fetch_k/metrics_3b_fetch_k_15.csv',
 'Experiments/3_retrieval_fetch_k/metrics_3c_fetch_k_10.csv',
 'Experiments/3_retrieval_fetch_k/metrics_3d_fetch_k_5.csv',
 'Experiments/3_retrieval_fetch_k/metrics_3e_fetch_k_3.csv']

In [22]:
# Here we find answers where the RAG chain refused to answer
for file_name in metrics_files:
    df = pd.read_csv(file_name, header=[0, 1], index_col=[0, 1])
    experiment_run = "_".join(os.path.basename(
        file_name).split("_")[1:]).split(".")[0]
    dataset = load_from_disk(os.path.join(
        experiment, "dataset_" + experiment_run))

    counter = 0
    exclude_list = []
    parties = list(dataset.keys())
    num_questions = len(dataset[parties[0]])
    for party in parties:
        for q_nr in range(24):
            answer = dataset[party][q_nr]["answer"]
            question = dataset[party][q_nr]["question"]

            if ("keine passende Antwort" in answer) and ("gefunden" in answer):
                counter += 1
                exclude_list.append([question, party])

    print(experiment_run)
    print(
        f"Fraction of None answers={len(exclude_list)/(len(parties)*num_questions):.3f}"
    )

3a_fetch_k_20
Fraction of None answers=0.326
3b_fetch_k_15
Fraction of None answers=0.319
3c_fetch_k_10
Fraction of None answers=0.326
3d_fetch_k_5
Fraction of None answers=0.250
3e_fetch_k_3
Fraction of None answers=0.243


In [23]:
metric_id = 0
df_plot = pd.DataFrame()
for file_name in metrics_files:
    df = pd.read_csv(file_name, header=[0, 1], index_col=[0, 1])
    metrics = list(set(df.columns.get_level_values(0)))
    experiment_run = "_".join(os.path.basename(file_name).split("_")[1:]).split(".")[0]
    df_plot[experiment_run] = df[metrics[metric_id]].mean(axis=0)


df = pd.DataFrame(df_plot).reset_index().rename(columns={"index": "party"})
df_melted = df.melt(id_vars=["party"], var_name="category", value_name="value")
print(metrics[metric_id])
print(df_melted.groupby("category")["value"].mean())
px.bar(
    df_melted,
    x="party",
    y="value",
    color="category",
    barmode="group",
    labels={"value": metrics[metric_id], "party": "Party"},
    title=f"{experiment}",
)

context_relevancy
category
3a_fetch_k_20    0.628217
3b_fetch_k_15    0.662543
3c_fetch_k_10    0.702022
3d_fetch_k_5     0.784698
3e_fetch_k_3     0.807645
Name: value, dtype: float64


In [5]:
metric_id = 1
df_plot = pd.DataFrame()
for file_name in metrics_files:
    df = pd.read_csv(file_name, header=[0, 1], index_col=[0, 1])
    metrics = list(set(df.columns.get_level_values(0)))
    experiment_run = "_".join(os.path.basename(file_name).split("_")[1:]).split(".")[0]
    df_plot[experiment_run] = df[metrics[metric_id]].mean(axis=0)


df = pd.DataFrame(df_plot).reset_index().rename(columns={"index": "party"})
df_melted = df.melt(id_vars=["party"], var_name="category", value_name="value")
print(metrics[metric_id])
print(df_melted.groupby("category")["value"].mean())
px.bar(
    df_melted,
    x="party",
    y="value",
    color="category",
    barmode="group",
    labels={"value": metrics[metric_id], "party": "Party"},
    title=f"{experiment}",
)

IndexError: list index out of range